***Install Important Libraries***

In [ ]:
!pip install transformers torch accelerate
!pip install -q huggingface_hub

**Gain access to Hugging face repository to use pretrained models like LLama and mistral**

In [ ]:
from huggingface_hub import login

login()

In [ ]:
from huggingface_hub import whoami
print(whoami()["name"])


NY0641


In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

llama_model = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(llama_model,use_auth_token=True) # use my huggingface token to use this model to convrt input into embeddings



config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Import the pipeline for predefined task performance

In [ ]:
from transformers import pipeline
llama_pipeline = pipeline("text-generation",model=llama_model,tokenizer=tokenizer,torch_dtype=torch.float16,device_map="auto")



Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Pipeline is setup, Now we need to create a chatbot using llama

In [ ]:
#Create a dictonary to hold reponses and add a system role for model clarity

messages = [{
    "role":"system",
    "content":"You are a helpful, smart and friendly AI assistant. Do not leave the sentence incomplete."
}]

In [ ]:
# create a function to get llama response

def get_llama_response(user_input: str) -> str: # Takes the user input and return a reply
  messages.append({"role":"user","content":user_input}) # save user input to dictionary

  #Convert chat into Llama format for chat models

  prompt = tokenizer.apply_chat_template(
      messages,
      tokenize=False,# telling llama not to give output as tokens, intead words
      add_generation_prompt=True # tells the model its assistant turn to reply
  )

  # generate response

  output = llama_pipeline(prompt,max_new_tokens=256,do_sample=True,temperature=0.7, top_k=5, top_p=0.9, eos_token_id = tokenizer.eos_token_id)

  full_text = output[0]["generated_text"]
  reply = full_text[len(prompt):].strip() # remove prompt from response
  # save assistant reply to memory
  messages.append({"role":"assistant","content":reply})
  return reply


In [42]:
# CLI loop
def run_chatbot():
  print("Llama chatbot is ready")

  while True:
    user_input = input("user: ")

    if user_input.lower() == "exit":
      print("chat ended.")
      break

    reply = get_llama_response(user_input)

    print("Chatbot: ",reply)

if __name__ == "__main__":
  run_chatbot()

Llama chatbot is ready
user: exit
chat ended.
